In [1]:
from keras.models import model_from_json

In [2]:
from inception_resnet_v1 import *
model = InceptionResNetV1()

In [3]:
import numpy as np
def l2_normalize(x):
    return x / np.sqrt(np.sum(np.multiply(x, x)))

def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [4]:
import numpy as np

from keras.models import Sequential
from keras.models import load_model
from keras.models import model_from_json
from keras.layers.core import Dense, Activation
from keras.utils import np_utils

from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input

import matplotlib.pyplot as plt
from keras.preprocessing import image

import pathlib
import pandas as pd


In [5]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(160, 160))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [6]:
BASE_PATH = pathlib.Path('../data/mfr2')
labels_path = BASE_PATH / 'mfr2_labels.txt'
pairs_path = BASE_PATH / 'pairs.txt'

In [7]:
# results = []


# unmasked_people = sorted(list(unmasked_dataset_path.iterdir()))
# for unmasked_person in unmasked_people:
#     masked_person = masked_dataset_path / unmasked_person.name
#     if not masked_person.is_dir():
#         continue
#     unmasked_image_path = str(sorted(list(unmasked_person.iterdir()))[0])
#     masked_image_path = str(sorted(list(masked_person.iterdir()))[0])
    
#     try:
#         unmasked_features = l2_normalize(model.predict(preprocess_image(last)))
#         masked_features = l2_normalize(model.predict(preprocess_image(masked_image_path)))

#         euclidean_distance = findEuclideanDistance(unmasked_features, masked_features)
#         matched = euclidean_distance < threshold
#         if matched:
#             print("verified... they are same person")
#         else:
#             print("unverified! they are not same person!")
#     except Exception as e:
#         print(e)
#         continue
    
#     results.append([unmasked_person.name, matched])

In [8]:
labels=pd.read_csv(filepath_or_buffer=str(labels_path))
labels=labels.apply(lambda x: x.apply(lambda y: y.strip() if type(y) == type('') else y), axis=0)
people=labels["person"].unique()

In [9]:
#np.array(labels['mask']=='no-mask') & np.array(labels['person']==people[0])
labels[np.array(labels['mask']=='no-mask') & np.array(labels['person']==people[0])]

,person,index,mask
1,AdrianDunbar,2,no-mask
3,AdrianDunbar,4,no-mask


In [10]:
with pairs_path.open() as f:
    pairs_data = [l.strip().split(' ') for l in f.readlines() if l.strip()]

pairs = []
for pair_data in pairs_data:
    if len(pair_data) == 3:
        pairs.append(((pair_data[0], pair_data[1]), (pair_data[0], pair_data[2])))
    elif len(pair_data) == 4:
        pairs.append(((pair_data[0], pair_data[1]), (pair_data[2], pair_data[3])))
    else:
        print('wtf', pair_data)

In [11]:
pairs[1]

(('ImranKhan', '2'), ('SimonCowell', '3'))

In [12]:
SAMPLE_SIZE = 10
threshold = 0.35

In [13]:
results = []

# positive test
for pair in pairs:
    (person1,index1),(person2,index2) = pair
    if person1 == person2:
    path1 = BASE_PATH / person1 / f'{person1}_{str(index1).rjust(4, "0")}.png'
    path2 = BASE_PATH / person2 / f'{person2}_{str(index2).rjust(4, "0")}.png'
    
    try:
        features1 = l2_normalize(model.predict(preprocess_image(path1)))
        features2 = l2_normalize(model.predict(preprocess_image(path2)))

        euclidean_distance = findEuclideanDistance(features1, features2)
        matched = euclidean_distance < threshold
        if matched:
            print("verified... they are same person")
        else:
            print("unverified! they are not same person!")
    except Exception as e:
        print(e)
        continue
    
    results.append([pair, matched, euclidean_distance])

IndentationError: expected an indented block (Temp/ipykernel_15036/2682737167.py, line 7)

In [ ]:
TP=len([x for x in results if x[1]]) / len(results)
FN=1-TP

In [ ]:
TP

In [ ]:
FN

In [ ]:
len(list(BASE_PATH.iterdir()))

In [ ]:
55**2

In [ ]:
results_neg = []
count = 0

for person1 in BASE_PATH.iterdir():
    for person2 in BASE_PATH.iterdir():
        if not person1.is_dir() or not person2.is_dir() or person1.name == person2.name or not person1.name.isalpha() or not person2.name.isalpha():
            continue
        count += 1
        if count >= len(results):
            break
        index1 = index2 = 1
        
        path1 = BASE_PATH / person1.name / f'{person1.name}_{str(index1).rjust(4, "0")}.png'
        path2 = BASE_PATH / person2.name / f'{person2.name}_{str(index2).rjust(4, "0")}.png'

        try:
            features1 = l2_normalize(model.predict(preprocess_image(path1)))
            features2 = l2_normalize(model.predict(preprocess_image(path2)))

            euclidean_distance = findEuclideanDistance(features1, features2)
            matched = euclidean_distance < threshold
            if matched:
                print("verified... they are same person")
            else:
                print("unverified! they are not same person!")
        except Exception as e:
            print(e)
            continue

        results_neg.append([pair, matched, euclidean_distance])

In [ ]:
FP=len([x for x in results_neg if x[1]]) / len(results_neg)
TN=1-TP

In [ ]:
FP

In [ ]:
TN

In [ ]:
TP=len([x for x in results if x[1]]) / (len(results) + len(results_neg))
FN=len([x for x in results if not x[1]]) / (len(results) + len(results_neg))
FP=len([x for x in results_neg if x[1]]) / (len(results) + len(results_neg))
TN=len([x for x in results_neg if not x[1]]) / (len(results) + len(results_neg))
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

array = [[TP,FN],
         [FP,TN]]

df_cm = pd.DataFrame(array, ['same', 'different'], ['verified', 'unverified'])
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

plt.show()

In [ ]:
from sklearn import svm
X = [x[2] for x in results] + [x[2] for x in results_neg]
y = [1 for x in results] + [0 for x in results_neg]
clf = svm.SVC()
clf.fit(np.array(X).reshape(-1, 1), y)

In [ ]:
ps=clf.predict(np.array([x[2] for x in results]).reshape(-1, 1))

In [ ]:
len([x for x in ps if x == 1])

In [ ]:
ns=clf.predict(np.array([x[2] for x in results_neg]).reshape(-1, 1))

In [ ]:
len([x for x in ps if x == 0])

In [ ]:
len(ps)

In [ ]:
TP=len([x for x in ps if x]) / (len(results) + len(results_neg))
FN=len([x for x in ps if not x]) / (len(results) + len(results_neg))
FP=len([x for x in ns if x]) / (len(results) + len(results_neg))
TN=len([x for x in ns if not x]) / (len(results) + len(results_neg))
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

array = [[TP,FN],
         [FP,TN]]

df_cm = pd.DataFrame(array, ['same', 'different'], ['verified', 'unverified'])
# plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

plt.show()

In [ ]:
TP

In [ ]:
FN

In [ ]:
FP

In [ ]:
len(results) + len(results_neg)

In [ ]:
import seaborn as sns
total_results = pd.DataFrame(data=[(x[2],"positive") for x in results]+[(x[2],"negative") for x in results_neg], columns=["distance", "type"])

In [ ]:
sns.histplot(total_results, x='distance', hue='type', element='step', kde=True, stat='percent')